In [50]:
import pandas as pd


df = pd.read_csv('data/538_highlight_results/test_00_00.csv')
df['region'] = df['username'].str.extract(r'user_(\d+)_')[0].astype(int)
score_cols = [col for col in df.columns if '_' in col and col != 'score']
best_score = -1
best_region = None
best_day = None
for region in df['region'].unique():
    df_region = df[df['region'] == region]
    days = df_region[score_cols].columns.str.split('_').str[0].unique()
    for day in days:
        day_cols = [col for col in score_cols if col.startswith(f'{day}_')]
        df_region = df_region.copy()
        df_region['day_score'] = df_region[day_cols].sum(axis=1)
        df_sorted = df_region.sort_values(['day_score', 'username'], ascending=[False, True])
        if len(df_sorted) >= 10:
            tenth = df_sorted.iloc[9]['day_score']
            if tenth > best_score:
                best_score = tenth
                best_region = region
                best_day = day
                
df_best = df[df['region'] == best_region]
day_cols = [col for col in score_cols if col.startswith(f'{best_day}_')]
df_best.loc[:, 'score'] = df_best[day_cols].sum(axis=1)
df_best_sorted = df_best.sort_values(['score', 'username'], ascending=[False, True])
output_cols = ['username'] + day_cols + ['score']
df_best_sorted = df_best_sorted[output_cols]
df_best_sorted.to_csv('data/538_highlight_results/result.csv', index=False, float_format='%.0f')